In [1]:
# import the required library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from scipy.stats.mstats import winsorize


In [34]:
# Load the csv of training file
file_path = 'assessment/train_df.csv' 
chunk_size = 10000  # small chunk size for small memory capacity

# Read in chunks
df_iterator = pd.read_csv(file_path, chunksize=chunk_size)

# Process chunks
data_frames = []
for i in df_iterator:
    data_frames.append(i)

# Concatenate all chunks into a single DataFrame
train_df = pd.concat(data_frames, ignore_index=True)

print(train_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351118 entries, 0 to 2351117
Data columns (total 46 columns):
 #   Column   Dtype  
---  ------   -----  
 0   id       int64  
 1   date     int64  
 2   c1       object 
 3   c2       object 
 4   country  object 
 5   c4       object 
 6   city     object 
 7   c6       object 
 8   c7       object 
 9   o1       int64  
 10  region   object 
 11  n1       float64
 12  n2       float64
 13  b1       float64
 14  n3       float64
 15  n4       float64
 16  n5       float64
 17  c9       object 
 18  hour     float64
 19  b2       float64
 20  b3       float64
 21  b4       float64
 22  b5       float64
 23  b6       float64
 24  b7       float64
 25  b8       float64
 26  b9       float64
 27  b10      float64
 28  b11      float64
 29  b12      float64
 30  n6       float64
 31  n7       float64
 32  n8       float64
 33  n9       float64
 34  n10      float64
 35  n11      float64
 36  n12      float64
 37  n13      float64
 38  n1

In [52]:
train_df.describe()

,id,date,o1,n1,n2,b1,n3,n4,n5,hour,...,n8,n9,n10,n11,n12,n13,n14,n15,n16,b17
count,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.351118e+06,2.334236e+06,...,2.244039e+06,2.315360e+06,2.258853e+06,2.284302e+06,2.263693e+06,2.295504e+06,2.235278e+06,2.318084e+06,1.228374e+06,2.351118e+06
mean,1.469592e+06,2.018391e+07,3.832843e+00,7.479608e-01,2.927694e+00,7.801599e-01,2.119779e+00,7.678921e+01,1.158218e+00,9.585255e+00,...,6.915920e-01,2.209306e+00,1.034298e+00,5.968571e-01,6.679488e-01,6.297646e-01,8.102580e+00,1.486383e+00,1.008311e+02,9.900822e-03
std,8.483559e+05,4.586940e+03,1.877755e+01,1.134143e+00,6.505663e+00,4.141383e-01,5.061158e+00,3.855962e+02,1.176902e+00,6.494604e+00,...,2.461087e+01,4.728596e+01,4.655253e+01,2.271428e+01,4.163010e+01,2.797213e+01,1.018222e+02,5.499303e+01,4.332954e+02,9.900909e-02
min,2.000000e+00,2.018010e+07,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.345795e+05,2.018052e+07,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.469616e+06,2.018093e+07,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,8.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.507000e+00,0.000000e+00
75%,2.204582e+06,2.019022e+07,2.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,7.000000e+00,1.000000e+00,1.400000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.924500e+01,0.000000e+00
max,2.938897e+06,2.019062e+07,5.328000e+03,1.245000e+03,1.350000e+03,1.000000e+00,1.350000e+03,5.960000e+04,1.262000e+03,2.300000e+01,...,9.341402e+03,9.744719e+03,2.652352e+04,7.439036e+03,3.542492e+04,5.747102e+03,5.960004e+04,4.633475e+04,5.045710e+04,1.000000e+00


In [53]:
# Get a list of categorical columns
categorical_cols = [col for col in train_df.columns if train_df[col].dtype == 'object']

# Iterate over categorical columns and print unique values
for col in categorical_cols:
    print(f"Unique values in '{col}' column:")
    print(train_df[col].value_counts())
    print("\n")

Unique values in 'c1' column:
c                          1010544
(n)                         442537
o                           391360
referral                    297846
listing                      40144
                            ...   
Remarketing-EU-Disp-2.1          1
dispmanagd-uruguay               1
Social                           1
Search-Greece                    1
main-page                        1
Name: c1, Length: 723, dtype: int64


Unique values in 'c2' column:
a    2000795
b     289069
c      61254
Name: c2, dtype: int64


Unique values in 'country' column:
United States                        668806
United Kingdom                       273017
India                                253596
Canada                                83052
Australia                             77586
                                      ...  
Montserrat                                2
St. Helena                                1
Niue                                      1
Falkland Islands (Isla

In [45]:
# Load the csv of prediction file
file_path = 'assessment/pred_df.csv' 
chunk_size = 10000  # small chunk size for small memory capacity

# Read in chunks
df_iterator = pd.read_csv(file_path, chunksize=chunk_size)


data_frames = []
for i in df_iterator:
    data_frames.append(i)

# Concatenate all into a single DataFrame
pred_df = pd.concat(data_frames, ignore_index=True)

print(pred_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587780 entries, 0 to 587779
Data columns (total 45 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       587780 non-null  int64  
 1   date     587780 non-null  int64  
 2   c1       587780 non-null  object 
 3   c2       587780 non-null  object 
 4   country  587780 non-null  object 
 5   c4       587780 non-null  object 
 6   city     587780 non-null  object 
 7   c6       587780 non-null  object 
 8   c7       587780 non-null  object 
 9   o1       587780 non-null  int64  
 10  region   587780 non-null  object 
 11  n1       587780 non-null  float64
 12  n2       587780 non-null  float64
 13  b1       587780 non-null  float64
 14  n3       587780 non-null  float64
 15  n4       587780 non-null  float64
 16  n5       587780 non-null  float64
 17  c9       314135 non-null  object 
 18  hour     583726 non-null  float64
 19  b2       583726 non-null  float64
 20  b3       583726 non-null  

In [35]:
train_df.head()

,id,date,c1,c2,country,c4,city,c6,c7,o1,...,n12,n13,n14,n15,n16,b13,b14,b15,b16,b17
0,1381034,20180418,helpdesk,a,United Kingdom,training,Stalybridge,training,(not set),4,...,0.0,0.0,NaN,0.0,3.804,NaN,NaN,NaN,NaN,0
1,471386,20180515,c,a,United States,(automatic matching),Santa Clara,google,Disp-US-CA-Top-Cities (Content Targeting),1,...,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0
2,1867433,20190508,c,a,United Kingdom,usdlt15k,London,google,sqa-youtube-uk,1,...,NaN,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0
3,2713558,20190325,search-philippines,a,Philippines,zendesk.com,Manila,google-adwords,search-philippines,1,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4,1545847,20190503,o,a,India,(not provided),Jamshedpur,google,(not set),1,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0


In [49]:
train_df['city'].value_counts()

(not set)      165265
London          87597
Bengaluru       41585
Chennai         40879
New York        36281
                ...  
Hollansburg         1
Oconee              1
Diboll              1
Macherio            1
Krasyliv            1
Name: city, Length: 24391, dtype: int64

In [54]:
train_df['o1'].value_counts()

1       1683264
2        255514
3         99409
4         57088
5         37875
         ...   
1167          1
626           1
1185          1
550           1
570           1
Name: o1, Length: 717, dtype: int64

In [46]:
pred_df.head()

,id,date,c1,c2,country,c4,city,c6,c7,o1,...,n11,n12,n13,n14,n15,n16,b13,b14,b15,b16
0,2882737,20190221,o,b,United States,(not provided),Goldenrod,google,(not set),1,...,0.0,0.0,0.0,1.625,0.0,1723.374,1.0,0.0,0.0,0.0
1,664701,20180107,o,a,New Zealand,(not provided),New Plymouth,google,(not set),1,...,0.0,0.0,0.0,NaN,0.0,8.832,NaN,NaN,NaN,NaN
2,714333,20181127,c,a,United States,USD40Kto60K,Newark,google,Disp-InMarket-Customsegment-US-Top-Cities,1,...,0.0,0.0,NaN,0.000,0.0,0.000,NaN,NaN,NaN,NaN
3,1248224,20190512,(n),a,Kenya,(not set),Nairobi,(direct),(not set),2,...,0.0,0.0,0.0,0.000,0.0,0.000,NaN,NaN,NaN,NaN
4,1973897,20180215,o,a,(not set),(not provided),(not set),google,(not set),1,...,0.0,0.0,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN


# delete columns
'c4': Drop this column due to the extremely large number of unique values (6679).

'c6': Drop this column as well due to the high number of unique values (9681).

'c9': Drop this column as it has 32589 unique values, which is computationally expensive to encode.

'city': Drop this column as it has 24391 unique values, which is quite high and may strain the available resources.

Columns to encode as categorical features:

'c1': Encode this column as it has a moderate number of unique values (723).

'c2': Encode this column as it has only 3 unique values, which is manageable.

'country': Encode this column as it has 238 unique values, which is a reasonable number.

'c7': Encode this column as it has 1714 unique values, which is manageable.

'region': Encode this column as it has 2251 unique values, which is a moderate number.

In [51]:
train_df['c7'].value_counts()

(not set)                                    976705
freemium                                     117777
Disp-AudienceKW-UK                            65909
Disp-AudienceKW-EU                            36931
Disp-US-CA-Top-Cities (Content Targeting)     29205
                                              ...  
inproduct-somethingstatus-referral                1
6 Steps to Handle Social Media Crises             1
inProduct-somethingmarketer-referral              1
Disp-HManagd-Italy                                1
bing-search-chile                                 1
Name: c7, Length: 1714, dtype: int64

In [42]:
train_df["n16"].value_counts()

0.000       543463
0.001          757
0.002          298
0.003          162
0.004          122
             ...  
187.797          1
284.286          1
880.438          1
150.340          1
1214.550         1
Name: n16, Length: 255392, dtype: int64

In [31]:
train_df.columns
train_df.isna().sum()* 100 / len(train_df)

id          0.000000
date        0.000000
c1          0.000000
c2          0.000000
country     0.000000
c4          0.000000
city        0.000000
c6          0.000000
c7          0.000000
o1          0.000000
region      0.000000
n1          0.000000
n2          0.000000
b1          0.000000
n3          0.000000
n4          0.000000
n5          0.000000
c9         46.561423
hour        0.718041
b2          0.718041
b3          0.718041
b4          0.718041
b5          0.718041
b6          0.718041
b7          0.718041
b8          0.718041
b9          0.718041
b10         0.718041
b11         0.718041
b12         0.718041
n6          2.374913
n7          1.505029
n8          4.554386
n9          1.520893
n10         3.924303
n11         2.841882
n12         3.718444
n13         2.365428
n14         4.927018
n15         1.405034
n16        47.753622
b13        92.589015
b14        92.589015
b15        92.589015
b16        92.589015
b17         0.000000
dtype: float64

In [40]:
train_df[['b2','b3','b4']].value_counts()

b2   b3   b4 
0.0  0.0  0.0    2117860
          1.0     106755
     1.0  0.0      56665
1.0  0.0  0.0      50372
          1.0       1417
0.0  1.0  1.0        677
1.0  1.0  0.0        481
          1.0          9
dtype: int64

In [38]:
# List of columns to drop beacuse there is 92.59% missing values
drop_columns = ['b13','b14','b15','b16']

# Drop columns using drop method
train_df.drop(drop_columns, axis=1,inplace=True)
train_df

,id,date,c1,c2,country,c4,city,c6,c7,o1,...,n8,n9,n10,n11,n12,n13,n14,n15,n16,b17
0,1381034,20180418,helpdesk,a,United Kingdom,training,Stalybridge,training,(not set),4,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,3.804,0
1,471386,20180515,c,a,United States,(automatic matching),Santa Clara,google,Disp-US-CA-Top-Cities (Content Targeting),1,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000,0
2,1867433,20190508,c,a,United Kingdom,usdlt15k,London,google,sqa-youtube-uk,1,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.000,0
3,2713558,20190325,search-philippines,a,Philippines,zendesk.com,Manila,google-adwords,search-philippines,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,1545847,20190503,o,a,India,(not provided),Jamshedpur,google,(not set),1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351113,565211,20180920,(n),b,United Kingdom,(not set),London,(direct),(not set),2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2351114,1160681,20190506,o,a,Germany,(not provided),Berlin,google,(not set),14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2351115,1935601,20180418,(n),a,United States,(not set),Bethlehem,(direct),(not set),1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2351116,2201174,20190206,(n),a,Netherlands,(not set),Soest,(direct),(not set),1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0


In [12]:
# handle binary variables value greater than 1 is replaced by 1
def binary_variables(df):
    binary_column = []
    for i in df.columns:
        if i.startswith('b'):
            binary_column.append(i)
    
    for i in binary_cols:
        df[i] = np.where(df[i] > 1, 1, df[i])  # replace the values grater than 1 with 1 else return same
        
    return df

In [ ]:
categorical_cols = [col for col in X.columns if col.startswith('c')]
ordinal_cols = [col for col in X.columns if col.startswith('o')]
numerical_cols = [col for col in X.columns if col.startswith('n')]
binary_cols = [col for col in df.columns if col.startswith('b') and col != 'b17']

In [44]:
# handling missing values
def handle_missing_values(df):
    # column types
    
    categorical_cols = [col for col in X.columns if col.startswith('c')]
    ordinal_cols = [col for col in X.columns if col.startswith('o')]
    numerical_cols = [col for col in X.columns if col.startswith('n')]
    binary_cols = [col for col in df.columns if col.startswith('b') and col != 'b17']

    
    # Remove 'Id' and 'b17' from numerical columns 
    if 'Id' in numerical_cols:
        numerical_cols.remove('Id')
    if 'b17' in numerical_cols:
        numerical_cols.remove('b17')
    
    # Create imputers
    numerical_imputer = SimpleImputer(strategy='mean')
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    ordinal_imputer = SimpleImputer(strategy='most_frequent')
    binary_imputer = SimpleImputer(strategy='constant', fill_value=0) # fill value is 0 because majority of data is 0 i.e.  2117860
    
    # c9 is object data type with 46 % of null value
    if 'c9' in categorical_cols:
        df['c9'].fillna('Unknown', inplace=True)  # Impute 'c9' with 'Unknown'
        categorical_cols.remove('c9')  # Remove 'c9' from categorical columns list
    
    # Apply imputers to appropriate columns
    df[numerical_cols] = numerical_imputer.fit_transform(df[numerical_cols])
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])
    df[ordinal_cols] = ordinal_imputer.fit_transform(df[ordinal_cols])
    df[binary_cols] = binary_imputer.fit_transform(df[binary_cols])
    
    return df


Handling Missing Values
Handling Categorical Data
Handling Numerical Data
normalisation



In [ ]:
# Handling outliers with Winsorization
def handle_outliers(df, columns):
    for col in columns:
        df[col] = winsorize(df[col], limits=[0.05, 0.05])  # Winsorize at 5th and 95th percentile
    return df

# numerical columns for outliers handling
numerical_cols = [col for col in train_df.columns if col.startswith('n')]
train_df = handle_outliers(train_df, numerical_cols)
pred_df = handle_outliers(pred_df, numerical_cols)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalization
def normalize_numeric(df, columns):
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

train_df = normalize_numeric(train_df, numerical_cols)
pred_df = normalize_numeric(pred_df, numerical_cols)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

def encode_categorical(df, categorical_cols):
    encoder = OneHotEncoder(drop='first', sparse=False)
    df_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))
    df_encoded.columns = encoder.get_feature_names(categorical_cols)
    df.drop(categorical_cols, axis=1, inplace=True)
    df = pd.concat([df, df_encoded], axis=1)
    return df

train_df = encode_categorical(train_df, categorical_cols)
pred_df = encode_categorical(pred_df, categorical_cols)


In [ ]:
X_train = train_df.drop(['Id', 'b17'], axis=1)
y_train = train_df['b17']
X_pred = pred_df.drop(['Id'], axis=1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Train the model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_pred)

# Prepare submission DataFrame
submission = pd.DataFrame({'id': pred_df['Id'], 'b17': y_pred})

# Save submission to CSV
submission.to_csv('submission.csv', index=False)
